In [1]:
import cdms2 as cdms
import MV2 as MV
import cdtime,cdutil,genutil
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import string
import glob
import scipy.stats as stats
# Local solution
# If running remotely, uncomment the following code:
# %%bash
# git clone https://github.com/katemarvel/CMIP5_tools
# import CMIP5_tools as cmip5
import sys,os
sys.path.append("/Users/kmarvel/Google Drive/python-utils")

import CMIP5_tools as cmip5
import DA_tools
import Plotting

from eofs.cdms import Eof
from eofs.multivariate.cdms import MultivariateEof
%matplotlib inline


In [2]:
NCA4regions={}
#Northwest (NW): (125°W–111°W, 42°N–49°N)
NCA4regions["NW"]=cdutil.region.domain(longitude=(-125,-111),latitude=(42,49))
#Southwest (SW): (124°W–102°W, 31°N–42°N)
NCA4regions["SW"]=cdutil.region.domain(longitude=(-124,-102),latitude=(31,42))
#Upper Great Plains (GPu): (116°W–95°W, 40°N–49°N)
NCA4regions["GPu"]=cdutil.region.domain(longitude=(-116,-95),latitude=(40,49))
#Lower Great Plains (GPl): (107°W–93°W, 26°N–40°N)
NCA4regions["GPl"]=cdutil.region.domain(longitude=(-107,-93),latitude=(26,40))
#Midwest (MW): (97°W–80°W, 36°N–50°N)
NCA4regions["MW"]=cdutil.region.domain(longitude=(-97,-80),latitude=(36,50))
#Northeast (NE): (82°W–67°W, 37°N–48°N)
NCA4regions["NE"]=cdutil.region.domain(longitude=(-82,-67),latitude=(37,48))
#Southeast (SE): (95°W–76°W, 25°N–39°N)
NCA4regions["SE"]=cdutil.region.domain(longitude=(-95,-76),latitude=(25,39))

#regions["central_plains"]=cdutil.region.domain(longitude=(-105,-92),latitude=(32,46))
#regions["southwest"]=cdutil.region.domain(longitude=(-125,-105),latitude=(32,41))


In [3]:
#sshfs mout to kdm2144@dester.ldeo.columbia.edu:/home/kdm2144/ 

#datadirec="/Volumes/RAVEN/CMIP6/cmip6rawyr/"
datadirec="/Users/kmarvel/Documents/DATA/dester/piControlRaw/"
writedirec="/Users/kmarvel/Documents/DATA/dester/regional_averages/NCA4/"
fixedvardirec="/Users/kmarvel/Documents/DATA/dester/fixedvar/"

In [4]:
#variables=[x.split("/")[-1] for x in glob.glob(datadirec+"*")]
variables=["mrsos","mrro"]

###### LOOP OVER ALL VARIABLES #####
for variable in variables:
    #print(variable)
    for region in NCA4regions.keys():
        cmd = "mkdir "+writedirec+"/"+region+"/"+variable
        os.system(cmd)

    
    #modeldirs=glob.glob(datadirec+experiment+"/"+variable+"/*")
    modeldirs=glob.glob(datadirec+variable+"/*")
    ### LOOP OVER ALL MODELS
    for direc in modeldirs:
        model=direc.split("/")[-1]
        #print(model)
        allfiles=glob.glob(direc+"/*"+variable+"*")
        experiments=np.unique([x.split(".")[1] for x in allfiles])
        ###### LOOP OVER ALL EXPERIMENTS #####
        for experiment in experiments:
            writedirecs={}
            for region in NCA4regions.keys():
            #print(experiment)
                region_writedirec=writedirec+region+"/"+variable+"/"+experiment+"/"
                cmd="mkdir "+region_writedirec
                os.system(cmd)
                writedirecs[region]=region_writedirec
                
            allfiles_experiment=glob.glob(direc+"/"+variable+"."+experiment+".*")
            rips=np.unique([x.split(".")[-3] for x in allfiles_experiment])

            landthresh=1
            #Get the land fraction
            landfiles=glob.glob(fixedvardirec+"sftlf*"+model+".*")
            

            if len(landfiles)==1:
                fland=cdms.open(landfiles[0])
                landfrac=fland("sftlf")
                fland.close()
            else:
                #print("can't find land fraction file for", model)
                #print(landfiles)
                continue
            #print(direc)



            ###### LOOP OVER ALL RIPS #####
            for rip in rips:
                #print(rip)
                writenames={}
                for key in NCA4regions.keys():
                    writename=writedirecs[key]+variable+"."+experiment+"."+model+"."+rip+".nc"
                    
#                     if (writename in glob.glob(writedirecs[key]+"*")):
#                         print("Already done ")
#                         continue
                    
                    writenames[key]=writename
                
                yearcheck=[]
                #ripfiles=sorted(glob.glob(direc+"/*"+rip+"*"))
                ripfiles=np.sort(glob.glob(direc+"/"+variable+"."+experiment+"."+model+"."+rip+"*"))
                print(variable+"."+experiment+"."+model+"."+rip+"*")
                L=len(ripfiles)
                i=0
                ripfile=ripfiles[i]
                frip=cdms.open(ripfile)
                data=frip(variable)
                frip.close()
                if data.shape[1:]!=landfrac.shape:
                    print ("land mask wrong shape for "+variable+"."+experiment+"."+model+"."+rip)
                    continue
                latax=landfrac.getLatitude()
                lonax=landfrac.getLongitude()
                tax=np.arange(12)
                fpath,fexpt,fmodel,frip,fyear,fnc=ripfile.split(".")
                ###Loop over regions
                for key in NCA4regions.keys():
                    region=NCA4regions[key]
                    DATA=MV.zeros(L*12)
                    #landdata=MV.masked_where(np.repeat(landfrac.asma()[np.newaxis],12,axis=0)<landthresh,data)
                    landdata=cmip5.cdms_clone(np.repeat(.01*landfrac.asma()[np.newaxis],12,axis=0)*data,data)
                    landdata.setAxis(1,latax)
                    landdata.setAxis(2,lonax)
                    #DATA[12*i:12*(i+1)]=cdutil.averager(landdata(region),axis='xy')
   
                    #yearcheck+=[float(fyear)]
                    for i in range(L):

                        ripfile=ripfiles[i]

                        f=cdms.open(ripfile)
                        data=f(variable)

                        #landdata=MV.masked_where(np.repeat(landfrac.asma()[np.newaxis],12,axis=0)<landthresh,data)
                        landdata=cmip5.cdms_clone(np.repeat(.01*landfrac.asma()[np.newaxis],12,axis=0)*data,data)
                        #Kludge since downloading process didn't preserve lat/lon designation
                        landdata.setAxis(1,latax)
                        landdata.setAxis(2,lonax)
                        f.close()

                        fpath,fexpt,fmodel,frip,fyear,fnc=ripfile.split(".")
                        DATA[12*i:12*(i+1)]=cdutil.averager(landdata(region),axis='xy')
    #                     CP[12*i:12*(i+1)]=cdutil.averager(landdata(central_plains),axis='xy')
    #                     SW[12*i:12*(i+1)]=cdutil.averager(landdata(southwest),axis='xy')
                        yearcheck+=[float(fyear)]
                    tax=cdms.createAxis(np.arange(L*12))
                    tax.designateTime()
                    tax.units='months since '+str(yearcheck[0])+'-1-1'
                    DATA.setAxis(0,tax)
                    DATA.id=variable
                    fw=cdms.open(writenames[key],"w")
                    fw.write(DATA)
                    fw.close()
    #                 CP.setAxis(0,tax)

    #                 SW.setAxis(0,tax)
    #                 CP.id=variable
    #                 SW.id=variable
    #                 cp_writename=cp_writedirec+variable+"."+experiment+"."+model+"."+rip+".nc"
    #                 fcpw=cdms.open(cp_writename,"w")    
    #                 fcpw.write(CP)
    #                 fcpw.close()
    #                 sw_writename=sw_writedirec+variable+"."+experiment+"."+model+"."+rip+".nc"
    #                 fsww=cdms.open(sw_writename,"w")    
    #                 fsww.write(SW)
    #                 fsww.close()  

mrsos.piControl.MIROC6.r1i1p1f1*


/Users/kmarvel/anaconda3/envs/cdat_plus/lib/python3.6/site-packages/numpy/ma/core.py:3169: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  dout = self.data[indx]
/Users/kmarvel/anaconda3/envs/cdat_plus/lib/python3.6/site-packages/numpy/ma/core.py:3201: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  mout = _mask[indx]


mrsos.piControl.UKESM1-0-LL.r1i1p1f2*
mrsos.piControl.CNRM-ESM2-1.r1i1p1f2*
mrsos.piControl.CESM2.r1i1p1f1*


KeyboardInterrupt: 